In [1]:
import sys, os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [2]:
sys.path.append(
    "/Users/wiegerscheurer/repos/physicspred"
)  # To enable importing from repository folders

In [3]:
150/2.2




68.18181818181817

In [5]:
from functions.physics import _rotate_90, _dir_to_vec, _vec_to_dir, _flip_dir
from functions.utilities import determine_sequence
from functions.analysis import (get_precision, 
                                get_data, 
                                get_false_negatives,
                                get_false_positives,
                                get_true_positives,
                                get_true_negatives,
                                filter_condition,
                                get_f1_score, 
                                get_rt, 
                                get_accuracy,
                                get_hit_rate,)


#### Acquire data per sub

In [36]:
all_sub_names = ["melvin", "paulo", "yifan", "ann", "qifei", "mingyao", "bilge", "yanni", "eva"]
sub_stack = pd.DataFrame()

for sub_idx, sub in enumerate(all_sub_names):
    this_sub = get_data(subject=f"sub-{sub}", task="ball_hue")
    sub_stack = pd.concat([sub_stack, this_sub])
    
melvin = get_data(subject="sub-melvin", task = "ball_hue")
yanni = get_data(subject="sub-yanni", task = "ball_hue")
ann = get_data(subject="sub-ann", task = "ball_hue")



sub_stack.reset_index(inplace=True, drop=True)
# sub_stack

In [37]:
yan_rt = get_rt(yanni, sim_con = False, expol_con = False, return_df = True)
yan_rt["rt"]

27          NaN
57     0.692383
67          NaN
72          NaN
91          NaN
96     0.696251
104    0.717886
105    0.694027
112    0.755391
113    0.629673
131         NaN
139         NaN
143    1.004873
Name: rt, dtype: float64

In [41]:
this_sub = random.choice(all_sub_names)
print(this_sub)
sub = get_data(subject=f"sub-{this_sub}", task="ball_hue")

simc_xpolc = get_rt(df=sub, sim_con=True, expol_con=True, return_df=True)["rt"]
simc_xpoli = get_rt(df=sub, sim_con=True, expol_con=False, return_df=True)["rt"]
simi_xpolc = get_rt(df=sub, sim_con=False, expol_con=True, return_df=True)["rt"]
simi_xpoli = get_rt(df=sub, sim_con=False, expol_con=False, return_df=True)["rt"]


all_kinds = [simc_xpolc, simc_xpoli, simi_xpolc, simi_xpoli]

for kind in all_kinds:
    print(len(kind))
    print(kind.mean(), kind.std())


qifei
13
0.621707153855823 0.06068163993488169
21
0.5726693145814352 0.0582362153535272
28
0.6291161826901281 0.07323958371855485
13
nan nan


In [56]:
simi_xpoli = get_rt(df=get_data(subject=f"sub-melvin", task="ball_hue"), sim_con=False, expol_con=False, return_df=True)["rt"]
simi_xpoli

13    NaN
87    NaN
89    NaN
94    NaN
95    NaN
97    NaN
125   NaN
Name: rt, dtype: float64

#### Integrate in pilot script

In [57]:
sim_cons = [True, False]
expol_cons = [True, False]
# randsub = random.choice(all_sub_names)
randsub = "eva"
print(f"Now looking at : {randsub}")
for sim_con in sim_cons:
    
    for expol_con in expol_cons:
        
        print(f"sim_con: {sim_con}, expol_con: {expol_con}")
        dat = get_data(subject=f"sub-{randsub}", task="ball_hue")
        dat_filt = dat[(dat["sim_congruent"] == sim_con) & (dat["abs_congruent"] == expol_con)]
        print(len(dat_filt))

Now looking at : eva
sim_con: True, expol_con: True
35
sim_con: True, expol_con: False
38
sim_con: False, expol_con: True
40
sim_con: False, expol_con: False
37


In [248]:
16 * 2

32

In [59]:
wie = get_data(subject="sub-eva", task="ball_hue")

n_targets = len(wie[wie["ball_change"] == 1]) # 20
n_nones = len(wie[wie["interactor"] == "none_d"])# 20
n_nones

# sort dataframe based on trials column
wie.sort_values(by="interactor", inplace=True)

wie["response"]

74      NaN
129    down
123     NaN
105    left
97      NaN
       ... 
3       NaN
20      NaN
112     NaN
46      NaN
0       NaN
Name: response, Length: 150, dtype: object

In [60]:

hit_rate = get_hit_rate(wie, False, False)
print(hit_rate)
wie_filt = filter_condition(wie, False, False)

fp = get_false_positives(wie_filt)
fn = get_false_negatives(wie_filt)
tp = get_true_positives(wie_filt)
tn = get_true_negatives(wie_filt)


print(f"False positives: {fp}")
print(f"False negatives: {fn}")
print(f"True positives: {tp}")
print(f"True negatives: {tn}")


hr = (tp) / (tp + fn)

hr
# get_misses(filter_condition(wie, False, False))

0.0
False positives: 0
False negatives: 18
True positives: 0
True negatives: 19


0.0

In [69]:
for sim_con in [True, False]:
    for xpol_con in [True, False]:
        print(f"sim_con: {sim_con}, xpol_con: {xpol_con}")
        print(f"{get_hit_rate(wie, sim_con, xpol_con):.2f}\n")

sim_con: True, xpol_con: True
0.68

sim_con: True, xpol_con: False
0.79

sim_con: False, xpol_con: True
0.56

sim_con: False, xpol_con: False
0.00



In [90]:
a = get_hit_rate(df=wie, sim_con=True, expol_con=True, return_df=True)["accuracy"]
a

0      False
1      False
14      True
18      True
25     False
31      True
33      True
48     False
61     False
71      True
72      True
77      True
81      True
86      True
88      True
92      True
96      True
107     True
121     True
137     True
144    False
146    False
Name: accuracy, dtype: object

In [86]:
for this_name in all_sub_names:
    wie = get_data(subject=f"sub-{this_name}", task="ball_hue")
    
    # Get hit rates for each condition
    simc_xpolc = get_hit_rate(df=wie, sim_con=True, expol_con=True, return_df=True)["accuracy"]
    simc_xpoli = get_hit_rate(df=wie, sim_con=True, expol_con=False, return_df=True)["accuracy"]
    simi_xpolc = get_hit_rate(df=wie, sim_con=False, expol_con=True, return_df=True)["accuracy"]
    simi_xpoli = get_hit_rate(df=wie, sim_con=False, expol_con=False, return_df=True)["accuracy"]

    print(f"Subject missed {get_false_negatives(wie)} trials")
    print(f"simc_xpolc: {simc_xpolc.mean()}, simc_xpoli: {simc_xpoli.mean()}, simi_xpolc: {simi_xpolc.mean()}, simi_xpoli: {simi_xpoli.mean()}")

    # Create data for the 2x2 heatmap
    z_values = [[simc_xpolc.mean(), simc_xpoli.mean()], 
                [simi_xpolc.mean(), simi_xpoli.mean()]]
    
    # Create a figure with subplot for heatmap
    fig = go.Figure()
    
    # Add heatmap
    fig.add_trace(go.Heatmap(
        z=z_values,
        x=['Congruent', 'Incongruent'],
        y=['Congruent', 'Incongruent'],
        text=[[f"{simc_xpolc.mean():.3f}<br>#trials: {len(simc_xpolc)}", f"{simc_xpoli.mean():.3f}<br>#trials: {len(simc_xpoli)}"], 
              [f"{simi_xpolc.mean():.3f}<br>#trials: {len(simi_xpolc)}", f"{simi_xpoli.mean():.3f}<br>#trials: {len(simi_xpoli)}"]],
        texttemplate="%{text}",
        textfont={"size":14},
        colorscale="RdGy",
        showscale=True,
        colorbar=dict(title="Hit Rate")
    ))
    
    # Update layout
    fig.update_layout(
        title=f"Hit Rate by Condition - Subject {this_name}",
        xaxis=dict(title="Motion Extrapolation", side="bottom"),
        yaxis=dict(title="Simulation"),
        width=600,
        height=500
    )
    
    # Show the figure
    fig.show()
    
    # Create a 2x2 bar plot alternative
    fig2 = go.Figure()
    
    # Define positions for the bars
    x_pos = [0, 1, 2, 3]
    x_labels = ['Sim C<br>Extrap C', 'Sim C<br>Extrap I', 'Sim I<br>Extrap C', 'Sim I<br>Extrap I']
    y_values = [simc_xpolc.mean(), simc_xpoli.mean(), simi_xpolc.mean(), simi_xpoli.mean()]
    
    # Add bars
    fig2.add_trace(go.Bar(
        x=x_pos,
        y=y_values,
        text=[f"{val:.3f}" for val in y_values],
        textposition='auto',
        marker_color=['rgba(0, 0, 255, 0.8)', 'rgba(0, 0, 200, 0.8)', 
                      'rgba(0, 0, 150, 0.8)', 'rgba(0, 0, 100, 0.8)'],
        name='Hit Rate'
    ))
    
    # Update layout
    fig2.update_layout(
        title=f"Hit Rate by Condition - Subject {this_name}",
        xaxis=dict(
            title="Conditions",
            tickmode='array',
            tickvals=x_pos,
            ticktext=x_labels
        ),
        yaxis=dict(title="Hit Rate"),
        width=600,
        height=500
    )
    
    # Show the figure
    fig2.show()

Subject missed 16 trials
simc_xpolc: 0.8461538461538461, simc_xpoli: 0.8620689655172413, simi_xpolc: 0.84, simi_xpoli: 0.0


Subject missed 17 trials
simc_xpolc: 0.8461538461538461, simc_xpoli: 0.84, simi_xpolc: 0.8571428571428571, simi_xpoli: 0.2222222222222222


Subject missed 14 trials
simc_xpolc: 1.0, simc_xpoli: 0.9166666666666666, simi_xpolc: 0.9583333333333334, simi_xpoli: 0.15384615384615385


Subject missed 25 trials
simc_xpolc: 0.6923076923076923, simc_xpoli: 0.8571428571428571, simi_xpolc: 0.7419354838709677, simi_xpoli: 0.0


Subject missed 16 trials
simc_xpolc: 1.0, simc_xpoli: 0.9523809523809523, simi_xpolc: 0.9285714285714286, simi_xpoli: 0.0


Subject missed 20 trials
simc_xpolc: 0.8888888888888888, simc_xpoli: 0.9166666666666666, simi_xpolc: 0.8928571428571429, simi_xpoli: 0.0


Subject missed 14 trials
simc_xpolc: 0.8571428571428571, simc_xpoli: 0.9565217391304348, simi_xpolc: 0.9642857142857143, simi_xpoli: 0.0


Subject missed 13 trials
simc_xpolc: 0.8571428571428571, simc_xpoli: 0.84, simi_xpolc: 0.9565217391304348, simi_xpoli: 0.5384615384615384


Subject missed 36 trials
simc_xpolc: 0.6818181818181818, simc_xpoli: 0.7894736842105263, simi_xpolc: 0.5625, simi_xpoli: 0.0


In [83]:

# Create an empty list to store all data frames
all_data = []

# Define types and their colors
types = ['Simcon + Xpolcon', 'Simcon + Xpolinc', 'Siminc + Xpolcon', 'Siminc + Xpolinc']

# Generate a colormap
cmap = plt.get_cmap('YlOrRd')  # You can choose a different colormap if you prefer
col_factor = 50
col_icept = 150
# Create the colors dictionary
type_colors = {name: f'rgba({int(cmap(i*col_factor + col_icept)[0]*255)}, {int(cmap(i*col_factor + col_icept)[1]*255)}, {int(cmap(i*col_factor + col_icept)[2]*255)}, 0.5)' 
          for i, name in enumerate(types)}

# Loop through all subjects (using the same structure as your original code)
for sub_name in all_sub_names:
# for sub_name in ["eva"]:
    sub = get_data(subject=f"sub-{sub_name}", task="ball_hue")
    
    simc_xpolc = get_rt(df=sub, sim_con=True, expol_con=True, return_df=True)["rt"]
    simc_xpoli = get_rt(df=sub, sim_con=True, expol_con=False, return_df=True)["rt"]
    simi_xpolc = get_rt(df=sub, sim_con=False, expol_con=True, return_df=True)["rt"]
    simi_xpoli = get_rt(df=sub, sim_con=False, expol_con=False, return_df=True)["rt"]
    
    # Create DataFrames without subject column - we'll aggregate across all subjects
    simc_xpolc_df = pd.DataFrame({
        'Reaction Time': simc_xpolc,
        'Type': 'Simcon + Xpolcon'
    })
    
    simc_xpoli_df = pd.DataFrame({
        'Reaction Time': simc_xpoli,
        'Type': 'Simcon + Xpolinc'
    })
    
    simi_xpolc_df = pd.DataFrame({
        'Reaction Time': simi_xpolc,
        'Type': 'Siminc + Xpolcon'
    })
    
    simi_xpoli_df = pd.DataFrame({
        'Reaction Time': simi_xpoli,
        'Type': 'Siminc + Xpolinc'
    })
    
    # Append all DataFrames to our list
    all_data.extend([simc_xpolc_df, simc_xpoli_df, simi_xpolc_df, simi_xpoli_df])

# Combine all the data into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Calculate mean reaction time for each type
type_means = combined_data.groupby('Type')['Reaction Time'].mean().to_dict()

# Create the plot
fig = go.Figure()

# Add violin plots for each type
for i, type_name in enumerate(types):
    subset = combined_data[combined_data['Type'] == type_name]
    
    fig.add_trace(go.Violin(
        x=[type_name] * len(subset),  # Use type name directly as x value
        y=subset['Reaction Time'],
        name=type_name,
        legendgroup=type_name,
        showlegend=True,
        box_visible=True,
        meanline_visible=True,
        points='all',
        jitter=0.2,
        pointpos=0.5,
        line_color=type_colors[type_name],
        side='negative',
        width=.5,
        spanmode='soft'
    ))

# Connect the means with a line
fig.add_trace(go.Scatter(
    x=types,
    y=[type_means[t] for t in types],
    mode='lines+markers',
    line=dict(color='rgba(0, 0, 0, 0.7)', width=8),
    marker=dict(
        size=14,  # Increased marker size
        color=[type_colors[t].replace('0.5', '1') for t in types],
        line=dict(color='black', width=4)
    ),
    name='Mean RT',
    hovertemplate='Mean: %{y:.3f}s<extra></extra>'
))

# Define a common font style for consistent text appearance
font_style = dict(
    family="Arial, sans-serif",
    size=20,  # Larger font size
    color="black"
)

# Update layout with enhanced text styling
fig.update_layout(
    title=dict(
        text="Average Reaction Times Across All Subjects\n\n",
        font=dict(size=25, family="Arial, sans-serif", color="black"),
        x=0.5,
        y=.985
    ),
    yaxis=dict(
        title=dict(
            text="Reaction Time (s)",
            font=font_style
        ),
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    xaxis=dict(
        title=dict(
            text="Response Type",
            font=font_style
        ),
        tickmode='array',
        tickvals=types,
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.01,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family="Arial, sans-serif"),
        borderwidth=2,
        bordercolor="White",
        title=dict(
            text="Response Type",
            font=dict(size=18, family="Arial, sans-serif")
        )
    ),
    violinmode='overlay',
    # height=1200,
    # width=1200,
    height=1200,
    width=1200,
    font=font_style,  # Default font for all other text elements
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for the mean values
for i, type_name in enumerate(types):
    fig.add_annotation(
        x=type_name,
        y=type_means[type_name],
        text=f"{type_means[type_name]:.3f}s",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,  # Slightly larger arrowhead
        arrowwidth=3,    # Thicker arrow
        arrowcolor='black',
        ax=120,
        ay=-90,
        font=dict(size=25, color='black', family="Arial, sans-serif")
    )

# Make the axis lines thicker
fig.update_xaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=False)
fig.update_yaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()




In [34]:

# Create an empty list to store all data frames
all_data = []

# Define types and their colors
types = ['Abstraction', 'Simulation', 'Both']

# Generate a colormap
cmap = plt.get_cmap('YlOrRd')  # You can choose a different colormap if you prefer
col_factor = 50
col_icept = 150
# Create the colors dictionary
type_colors = {name: f'rgba({int(cmap(i*col_factor + col_icept)[0]*255)}, {int(cmap(i*col_factor + col_icept)[1]*255)}, {int(cmap(i*col_factor + col_icept)[2]*255)}, 0.5)' 
          for i, name in enumerate(types)}

# Loop through all subjects (using the same structure as your original code)
for sub_name in all_sub_names:
    sub = get_data(subject=f"sub-{sub_name}", task="ball_hue")
    # Get your data as before
    rt_dubs = get_rt(sub, hypothesis="both", include_dubtrials="only", return_df=True)
    rt_separate = get_rt(sub, hypothesis="both", include_dubtrials=False, return_df=True)
    sim_rt = rt_separate["simulation"]["rt"]
    abs_rt = rt_separate["abstraction"]["rt"]
    both_rt = rt_dubs["sim + abs"]["rt"]
    
    # Create DataFrames without subject column - we'll aggregate across all subjects
    sim_df = pd.DataFrame({
        'Reaction Time': sim_rt,
        'Type': 'Simulation'
    })
    
    abs_df = pd.DataFrame({
        'Reaction Time': abs_rt,
        'Type': 'Abstraction'
    })
    
    both_df = pd.DataFrame({
        'Reaction Time': both_rt,
        'Type': 'Both'
    })
    
    # Append all DataFrames to our list
    all_data.extend([abs_df, sim_df, both_df])

# Combine all the data into a single DataFrame
combined_data = pd.concat(all_data, ignore_index=True)

# Calculate mean reaction time for each type
type_means = combined_data.groupby('Type')['Reaction Time'].mean().to_dict()

# Create the plot
fig = go.Figure()

# Add violin plots for each type
for i, type_name in enumerate(types):
    subset = combined_data[combined_data['Type'] == type_name]
    
    fig.add_trace(go.Violin(
        x=[type_name] * len(subset),  # Use type name directly as x value
        y=subset['Reaction Time'],
        name=type_name,
        legendgroup=type_name,
        showlegend=True,
        box_visible=True,
        meanline_visible=True,
        points='all',
        jitter=0.2,
        pointpos=0.5,
        line_color=type_colors[type_name],
        side='negative',
        width=.5,
        spanmode='soft'
    ))

# Connect the means with a line
fig.add_trace(go.Scatter(
    x=types,
    y=[type_means[t] for t in types],
    mode='lines+markers',
    line=dict(color='rgba(0, 0, 0, 0.7)', width=8),
    marker=dict(
        size=14,  # Increased marker size
        color=[type_colors[t].replace('0.5', '1') for t in types],
        line=dict(color='black', width=4)
    ),
    name='Mean RT',
    hovertemplate='Mean: %{y:.3f}s<extra></extra>'
))

# Define a common font style for consistent text appearance
font_style = dict(
    family="Arial, sans-serif",
    size=20,  # Larger font size
    color="black"
)

# Update layout with enhanced text styling
fig.update_layout(
    title=dict(
        text="Average Reaction Times Across All Subjects\n\n",
        font=dict(size=25, family="Arial, sans-serif", color="black"),
        x=0.5,
        y=.985
    ),
    yaxis=dict(
        title=dict(
            text="Reaction Time (s)",
            font=font_style
        ),
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    xaxis=dict(
        title=dict(
            text="Response Type",
            font=font_style
        ),
        tickmode='array',
        tickvals=types,
        tickfont=font_style,
        tickwidth=2,
        showline=True,
        linewidth=3,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.01,
        xanchor="center",
        x=0.5,
        font=dict(size=18, family="Arial, sans-serif"),
        borderwidth=2,
        bordercolor="White",
        title=dict(
            text="Response Type",
            font=dict(size=18, family="Arial, sans-serif")
        )
    ),
    violinmode='overlay',
    # height=1200,
    # width=1200,
    height=800,
    width=800,
    font=font_style,  # Default font for all other text elements
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for the mean values
for i, type_name in enumerate(types):
    fig.add_annotation(
        x=type_name,
        y=type_means[type_name],
        text=f"{type_means[type_name]:.3f}s",
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,  # Slightly larger arrowhead
        arrowwidth=3,    # Thicker arrow
        arrowcolor='black',
        ax=120,
        ay=-90,
        font=dict(size=25, color='black', family="Arial, sans-serif")
    )

# Make the axis lines thicker
fig.update_xaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=False)
fig.update_yaxes(mirror=True, ticks='outside', tickwidth=3, ticklen=10, showgrid=True, gridwidth=1, gridcolor='lightgray')

fig.show()